# do note use this, use the python program (.py)

In [2]:
import numpy as np
import pandas as pd
import astroquery
import matplotlib.pyplot as plt
import glob

import matplotlib

from astropy.coordinates import SkyCoord
from astropy import units as u
import tqdm

%matplotlib inline

In [3]:
fn = '/Users/tom/Dropbox/TIC5/ctl.csv'

header = [
    'RA','DEC',
    'ECLONG','ECLAT', 'V', 'Ks',
    'TESSMAG','TEFF','RADIUS','MASS','CONTRATIO','PRIORITY',
    ]
usecols = [13, 14, 30, 60, 64, 87, 70, 72, 84, 26, 27, 46]
df = pd.read_csv(fn, names=header, usecols=usecols)

In [7]:
selectnum = int(np.round(110000/26., ))
df.loc[:,'SELECTED'] = np.zeros(df.shape[0], dtype='bool')

eloop = np.linspace(0,360,14)
espace = np.diff(eloop)[0] # get space in degrees
for elon in eloop[:-1]:
    # southern hemisphere
    ms = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT <= -6)
    selectthese = df.loc[ms].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     # northern hemisphere
    mn = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT >= 6)
    selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     print(df.loc[df.SELECTED == True].shape[0])
    print(elon,elon+espace,df.loc[df.SELECTED == True].shape[0])

0.0 27.6923076923 8462
27.6923076923 55.3846153846 16924
55.3846153846 83.0769230769 25386
83.0769230769 110.769230769 33848
110.769230769 138.461538462 42310
138.461538462 166.153846154 50772
166.153846154 193.846153846 59234
193.846153846 221.538461538 67696
221.538461538 249.230769231 76158
249.230769231 276.923076923 84620
276.923076923 304.615384615 93082
304.615384615 332.307692308 101544
332.307692308 360.0 110006


In [8]:
selected = df[df.SELECTED == True]

In [9]:
selected.to_csv('../data/selected5.csv.bz2',compression='bz2')

# ok let's now try to select things kinda realistically

In [10]:
fn = '/Users/tom/Dropbox/TIC5/ctl.csv'

header = [
    'RA','DEC',
    'ECLONG','ECLAT', 'V', 'Ks',
    'TESSMAG','TEFF','RADIUS','MASS','CONTRATIO','PRIORITY',
    ]
usecols = [13, 14, 30, 60, 64, 87, 70, 72, 84, 26, 27, 46]
df = pd.read_csv(fn, names=header, usecols=usecols)

In [11]:
# have 7000 targets a each of the poles
# 4000 in other fields
selectnum = 4000
selectpoles = 7000
df.loc[:,'SELECTED'] = np.zeros(df.shape[0], dtype='bool')

eloop = np.linspace(0,360,14)
espace = np.diff(eloop)[0] # get space in degrees

#poles first
mn = (df.ECLAT >= 78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

mn = (df.ECLAT <= -78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

for elon in eloop[:-1]:
    # southern hemisphere
    ms = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT <= -6) & (df.ECLAT > -78)
    selectthese = df.loc[ms].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     # northern hemisphere
    mn = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT >= 6) & (df.ECLAT < 78)
    selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
    print(elon,elon+espace,df.loc[df.SELECTED == True].shape[0])

0.0 27.6923076923 22000
27.6923076923 55.3846153846 30000
55.3846153846 83.0769230769 38000
83.0769230769 110.769230769 46000
110.769230769 138.461538462 54000
138.461538462 166.153846154 62000
166.153846154 193.846153846 70000
193.846153846 221.538461538 78000
221.538461538 249.230769231 86000
249.230769231 276.923076923 94000
276.923076923 304.615384615 102000
304.615384615 332.307692308 110000
332.307692308 360.0 118000


In [12]:
# now add tvguide cameras
df['tess_ccd'] = np.zeros(df.shape[0], dtype='int')
df['obs_len'] = np.zeros(df.shape[0], dtype='float') 
for i in tqdm(df.index):
    #need a hack to force stars in the northern hemisphere to work
    if df.loc[i,'ECLAT'] >= 6:
        gc = SkyCoord(lon=df.loc[i,'ECLONG'] * u.degree, lat=df.loc[i, 'ECLAT'] * u.degree * -1,
                      frame='barycentrictrueecliptic')
        obj = TessPointing(gc.icrs.ra.value, gc.icrs.dec.value)
    else:
        obj = TessPointing(df.loc[i,'RA'], df.loc[i,'DEC'])
    df.loc[i,'tess_ccd'] = obj.get_camera()
    df.loc[i,'obs_len'] = obj.get_13cameras()[obj.get_13cameras() > 0].shape[0] * 27.4

NameError: name 'tqdm' is not defined

In [ ]:
selected = df[df.SELECTED == True]

In [ ]:
selected.to_csv('../data/selectedreal5.csv.bz2',compression='bz2')

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u

In [ ]:
gc = SkyCoord(ra=150*u.degree, dec=45*u.degree, frame='icrs')

In [ ]:
gc.barycentrictrueecliptic.lon.value

# now do our alternative with 9,000 at the poles and 3,000 in each sector

In [ ]:
fn = '/Users/tom/Dropbox/TIC5/ctl.csv'

header = [
    'RA','DEC',
    'ECLONG','ECLAT', 'V', 'Ks',
    'TESSMAG','TEFF','RADIUS','MASS','CONTRATIO','PRIORITY',
    ]
usecols = [13, 14, 30, 60, 64, 87, 70, 72, 84, 26, 27, 46]
df = pd.read_csv(fn, names=header, usecols=usecols)

In [ ]:


selectnum = 2000
selectpoles = 9000
df.loc[:,'SELECTED'] = np.zeros(df.shape[0], dtype='bool')

eloop = np.linspace(0,360,14)
espace = np.diff(eloop)[0] # get space in degrees

#poles first
mn = (df.ECLAT >= 78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

mn = (df.ECLAT <= -78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

for elon in eloop[:-1]:
    # southern hemisphere
    ms = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT <= -6) & (df.ECLAT > -78)
    selectthese = df.loc[ms].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     # northern hemisphere
    mn = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT >= 6) & (df.ECLAT < 78)
    selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
    print(elon,elon+espace,df.loc[df.SELECTED == True].shape[0])

In [4]:
selected = df[df.SELECTED == True]

In [5]:
selected.to_csv('../data/selectedreal5-LongerBaseline.csv.bz2',compression='bz2')